In [1]:
!git clone https://github.com/google/flatbuffers.git
!cd flatbuffers && cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release && make -j

!cd flatbuffers/python && python3 setup.py install

Cloning into 'flatbuffers'...
Updating files:  19% (233/1171)
Updating files:  20% (235/1171)
Updating files:  21% (246/1171)
Updating files:  22% (258/1171)
Updating files:  23% (270/1171)
Updating files:  24% (282/1171)
Updating files:  25% (293/1171)
Updating files:  26% (305/1171)
Updating files:  27% (317/1171)
Updating files:  28% (328/1171)
Updating files:  29% (340/1171)
Updating files:  30% (352/1171)
Updating files:  31% (364/1171)
Updating files:  31% (373/1171)
Updating files:  32% (375/1171)
Updating files:  33% (387/1171)
Updating files:  34% (399/1171)
Updating files:  35% (410/1171)
Updating files:  36% (422/1171)
Updating files:  37% (434/1171)
Updating files:  38% (445/1171)
Updating files:  39% (457/1171)
Updating files:  40% (469/1171)
Updating files:  41% (481/1171)
Updating files:  42% (492/1171)
Updating files:  43% (504/1171)
Updating files:  43% (508/1171)
Updating files:  44% (516/1171)
Updating files:  45% (527/1171)
Updating files:  46% (539/1171)
Updating f

In [2]:
!wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!alias python=python3
!./flatbuffers/flatc --python ./schema.fbs

'alias' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!wget https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/palm_detection.tflite

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
import tflite

In [14]:
with open('../models/palm_detection_lite.tflite', 'rb') as f:
        buf = f.read()
        model = tflite.Model.GetRootAsModel(buf, 0)


In [17]:
graph = model.Subgraphs(0)

In [35]:
graph.Operators(0).OpcodeIndex()

9

In [ ]:
tflite.BuiltinOperator.

In [3]:
import os
import numpy as np
from collections import OrderedDict
from tensorflow.lite.python.interpreter import Interpreter

In [6]:
model = Interpreter("../models/palm_detection_finetuned.tflite", 0)

In [7]:

for t in model.get_tensor_details():
    name = t['name'].replace('model_1/model/', '')
    print("%3d %150s %20s %s" % (t['index'], name, t['shape'], t['dtype']))

  0                                                                                                                                                    445        [ 1 64 64 64] <class 'numpy.float32'>
  1                                                                                                                                                  r16.1        [  1 512  18] <class 'numpy.float32'>
  2                                                                                                                                  input.53_te_transform    [  1  32  32 128] <class 'numpy.float32'>
  3                                                                                                                                            661_te_attr                  [4] <class 'numpy.int32'>
  4                                                                                                                                       561_te_transform    [  1  32  32 128] <class 'numpy.float32'>
  

In [8]:
model.allocate_tensors()

In [9]:
model.get_tensor(17).shape

(256,)

In [10]:
model.get_input_details()

[{'name': 'x.3',
  'index': 159,
  'shape': array([  1, 256, 256,   3]),
  'shape_signature': array([  1, 256, 256,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [11]:
model.get_output_details()

[{'name': '121',
  'index': 221,
  'shape': array([   1, 2944,   18]),
  'shape_signature': array([   1, 2944,   18]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': '116',
  'index': 249,
  'shape': array([   1, 2944,    2]),
  'shape_signature': array([   1, 2944,    2]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [84]:
anchors = np.load('anchors.npy')
print(np.min(anchors))
print(np.max(anchors))

0.015625
1.0


In [4]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

In [5]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

185

In [6]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 0)  # FLOAT32
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float32)
    W = W.reshape(shape)
    return W
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
W.shape, b.shape

((32, 3, 3, 3), (32,))

In [7]:
import torch
from blazepalm import PalmDetector

In [8]:
net = PalmDetector()
net

PalmDetector(
  (backbone1): Sequential(
    (0): ConstantPad2d(padding=(0, 1, 0, 1), value=0.0)
    (1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
    (2): ReLU(inplace=True)
    (3): ResBlock(
      (f): Sequential(
        (0): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
        (1): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
        (2): ResModule(
          (convs): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )

In [9]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 0:
        probable_names.append(tensor.Name().decode("utf-8"))
        
probable_names[:5]

['conv2d/Kernel',
 'conv2d/Bias',
 'depthwise_conv2d/Kernel',
 'depthwise_conv2d/Bias',
 'conv2d_1/Kernel']

In [10]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    convert[name] = probable_names[i]
    i += 1

In [11]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:
        if W.shape[0] == 1:
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
          if "transpose" in src:
            W = W.transpose((3, 0, 1, 2)) # transpose
          else:
            W = W.transpose((0, 3, 1, 2))  # regular conv
    
    new_state_dict[dst] = torch.from_numpy(W)


backbone1.1.weight conv2d/Kernel (32, 3, 3, 3) torch.Size([32, 3, 3, 3])
backbone1.1.bias conv2d/Bias (32,) torch.Size([32])
backbone1.3.f.0.convs.0.weight depthwise_conv2d/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.0.convs.0.bias depthwise_conv2d/Bias (32,) torch.Size([32])
backbone1.3.f.0.convs.1.weight conv2d_1/Kernel (32, 1, 1, 32) torch.Size([32, 32, 1, 1])
backbone1.3.f.0.convs.1.bias conv2d_1/Bias (32,) torch.Size([32])
backbone1.3.f.1.convs.0.weight depthwise_conv2d_1/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.1.convs.0.bias depthwise_conv2d_1/Bias (32,) torch.Size([32])
backbone1.3.f.1.convs.1.weight conv2d_2/Kernel (32, 1, 1, 32) torch.Size([32, 32, 1, 1])
backbone1.3.f.1.convs.1.bias conv2d_2/Bias (32,) torch.Size([32])
backbone1.3.f.2.convs.0.weight depthwise_conv2d_2/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.3.f.2.convs.0.bias depthwise_conv2d_2/Bias (32,) torch.Size([32])
backbone1.3.f.2.convs.1.weight conv2d_3/Kernel (32

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  app.launch_new_instance()


backbone2.1.f.0.convs.0.bias depthwise_conv2d_24/Bias (256,) torch.Size([256])
backbone2.1.f.0.convs.1.weight conv2d_25/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.0.convs.1.bias conv2d_25/Bias (256,) torch.Size([256])
backbone2.1.f.1.convs.0.weight depthwise_conv2d_25/Kernel (1, 3, 3, 256) torch.Size([256, 1, 3, 3])
backbone2.1.f.1.convs.0.bias depthwise_conv2d_25/Bias (256,) torch.Size([256])
backbone2.1.f.1.convs.1.weight conv2d_26/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.1.convs.1.bias conv2d_26/Bias (256,) torch.Size([256])
backbone2.1.f.2.convs.0.weight depthwise_conv2d_26/Kernel (1, 3, 3, 256) torch.Size([256, 1, 3, 3])
backbone2.1.f.2.convs.0.bias depthwise_conv2d_26/Bias (256,) torch.Size([256])
backbone2.1.f.2.convs.1.weight conv2d_27/Kernel (256, 1, 1, 256) torch.Size([256, 256, 1, 1])
backbone2.1.f.2.convs.1.bias conv2d_27/Bias (256,) torch.Size([256])
backbone2.1.f.3.convs.0.weight depthwise_conv2d_27/Kernel (1, 3, 3, 256) t

In [12]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [13]:
torch.save(net.state_dict(), "palmdetector.pth")